## Check approximate quantum circuit versus Hamiltonian in form of Pauli strings

### First load required libraries and define some useful functions:

In [1]:
import numpy as np
from scipy.linalg import expm
from numpy.linalg import eigvals
from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import SparsePauliOp, Operator
from qiskit.opflow import PrimitiveOp, PauliTrotterEvolution


def char_to_pauli(char):
    """Returns the Pauli matrix for a given character in I, X, Y, Z"""
    char = char.upper() # convert lower to upper case
    if char == 'I':
        return np.matrix([[1,0],[0,1]])
    if char == 'X':
        return np.matrix([[0,1],[1,0]])
    if char == 'Y':
        return np.matrix([[0,-1j],[1j,0]])
    if char == 'Z':
        return np.matrix([[1,0],[0,-1]])
    raise Exception("Character '" + char + "' does not correspond to Pauli matrix")

    
def string_to_operator(string):
    """Calculates the operator (Kronecker product) from a Pauli string"""
    op = np.array(1, dtype=np.complex128)
    for c in string:
        op = np.kron(op, char_to_pauli(c))
    return op


def pauli_unitary(string, coefficient):
    """Calculates the unitary matrix exp(-icH) for a given Pauli string H with coefficient c"""
    return expm(-1j * coefficient * string_to_operator(string))


def pauli_circuit(string, coefficient):
    """Quantum circuit for the given Pauli string with coefficient, i.e. implementation of exp(-icH)"""
    op = PrimitiveOp(SparsePauliOp(string, coefficient))
    unitary = PauliTrotterEvolution().convert(op.exp_i())
    return unitary.to_circuit().decompose()


def error(a, b):
    """Error between two equal sized unitaries a, b defined as
       largest magnitude eigenvalue of a-b after minimizing over all relative phases
    """
    return np.max(np.abs(eigvals(a-b)))


class PauliHamiltonian:
    """Class for Hamiltonian built from Pauli strings
    Attributes:
        operators: list of Pauli strings
        coefficients: dictionary {'Pauli String': coefficient}
        qubits: number of qbits 
    """
    
    def __init__(self, operators = [], coefficients = {}):
        nq = 0
        if len(operators):
            nq = len(operators[0])
            for o in operators:
                if len(o) != nq:
                    raise Exception("Operator length mismatch")
        self.operators = list(operators)
        self.coefficients = {}
        for o in self.operators:
            if o in coefficients.keys():
                self.coefficients[o] = coefficients[o]
            else:
                self.coefficients[o] = 0
        self.qubits = nq
        self.__matrix = None
        self.__unitary = None
    
    def dim(self):
        """Matrix dimension 2^n with n qubits"""
        return 2**self.qubits
    
    def matrix(self):
        """Hamiltonian in matrix form"""
        if self.__matrix is None: # matrix form has not been calculated yet
            n = self.dim()
            self.__matrix = np.empty((n, n), dtype=np.complex128)
            for o in self.operators:
                self.__matrix += self.coefficients[o] * string_to_operator(o)
        return self.__matrix
    
    def unitary(self):
        """exp(-iH) in matrix form"""
        if self.__unitary is None: # unitary has not been calculated yet
            self.__unitary = expm(-1j * self.matrix())
        return self.__unitary
    
    def global_phase(self):
        """Prefactor exp(-ic) of global phase, c = coefficient of III...I"""
        if self.qubits:
            if 'I'*self.qubits in self.coefficients.keys():
                return np.exp(-1j*self.coefficients['I'*self.qubits])
        return 1
    
    def circuit(self):
        """Transpiled quantum circuit that implements the unitary operator product
           exp(-i c1 O1) exp(-i c1 O1) ... exp(-i cn On)
           where O1,...,On are the Pauli operators in self.operators
           and c1,...,cn the corresponding coefficients as in self.coefficients
           NOTE: this is NOT an exact implementation of exp(-iH) but rather a
           Trotterization with a single Trotter step
        """
        ops = self.operators
        if len(ops) < 1:
            return 0
        qc = QuantumCircuit(self.qubits)
        # go through ops in reverse order to built circuit with rightmost unitary first
        for i in range(len(ops)-1,-1,-1):
            if ops[i] != 'I'*self.qubits: # ignore identity (yields global phase)
                qc.append(pauli_circuit(ops[i], self.coefficients[ops[i]]),range(self.qubits))
        return transpile(qc, basis_gates=['u','cx'])

    
def hamiltonian_from_file(filename):
    """Read Hamiltonian from file and return PauliHamiltonian object.
    File format: 'coefficient Pauli-string' per line, separated by space"""
    try:
        file = open(filename) # read file
    except:
        print("An exception occurred trying to read the file: " + filename) 
    lines = file.read().split('\n')
    file.close()
    h = PauliHamiltonian() # create object to return
    for l in lines:
        if len(l): # exclude empty lines
            [c,o] = l.split() # split in coefficient c and operator string o
            if h.qubits:
                if len(o) != h.qubits: # Make sure all operators have the same dimension
                    raise Exception("Operator string '" + o + "' does not match dimension (" + h.qubits + ")")
            else:
                h.qubits = len(o) # set no. qubits to length of operator string
            h.operators.append(o) # add operator to list of operators of object h
            h.coefficients[o] = float(c) # add to the dictionary of coefficients for h
    return h


### Generate the quantum circuit approximating our Hamiltonian

The Hamiltonian is given in **hamiltonian.txt** in the form *+0.003034656830204855 IIIYYIIIYY ...* (one expression per line).

We also load the Hamiltonian in **h_approx.txt** which is the result of our optimization procedure.

The idea is to generate a short circuit - as a single Trotter step - from this approximated Hamiltonian.

We show that this circuit is the one stored in **circuit.txt** as OPENQASM 2.0 file.

In [2]:
# Load Hamiltonian:
h = hamiltonian_from_file('hamiltonian.txt')

# Load approximate Hamiltonian and get quantum circuit (qiskit.QuantumCircuit object):
ha = hamiltonian_from_file('h_approx.txt')
qc = ha.circuit()
print('Generated a quantum circuit of depth: ', qc.depth())

# Confirm that generated circuit matches the one stored in circuit.txt
if qc.qasm() == open('circuit.txt').read():
    print('Quantum circuit matches circuit.txt')
else:
    print('Mismatch!')

Generated a quantum circuit of depth:  781
Quantum circuit matches circuit.txt


### Now check our circuit versus Hamiltonian

We calculate the error between the unitary corresponding to the generated quantum circuit and the unitary exp(-iH) of the *full* Hamiltonian in **hamiltonian.txt**

In [3]:
# Calculate unitary exp(-iH):
hu = h.unitary()

# Calculate unitary for the quantum circuit:
cu = Operator(qc).data
# Correct for the global phase of the circuit:
cu *= h.global_phase()

# Calculate the error between both unitaries:
e = error(hu, cu)
print('The error is: ', e)

The error is:  0.0989122606998372


**The error satisfies the requirement e < 0.1**

Hence, we have a quantum circuit of depth 781 that approximates exp(-iH) with an error below 0.1 as required.


### Now we try to confirm this error directly between the QASM file and the full Hamiltonian

In [4]:
# Load circuit from QASM file:
qc = QuantumCircuit.from_qasm_file('circuit.txt')
print('Loaded circuit of depth: ', qc.depth())

# Repeat the error calculation as before:
cu = Operator(qc).data
cu *= h.global_phase()
e = error(hu, cu)
print('The error is: ', e)

Loaded circuit of depth:  781
The error is:  0.28856775845567


As we see, loading the circuit from the QASM file results in a larger error of 0.29 - what went wrong?

Try to find the true error - disregarding global phase - by minimizing over all global phase values:

In [5]:
from scipy.optimize import minimize

# Minimazation function: error between unitaries with relative phase p
def f(p):
    return error(hu, np.exp(-1j*p)*cu)

m = minimize(f, 0, method='COBYLA', tol=0.0001)
print('Found minimum error: ', m.fun)
print('...for relative phase ', m.x)

Found minimum error:  0.09890003316813079
...for relative phase  [-0.2174293]


When minimizing, we again find the error of 0.099 < 0.1, satisfying the requirements.

Weirdly, Qiskit seems to add a seemingly random relative phase when re-loading the QASM file previously saved.